# 論文 5：透過最小化描述長度保持神經網路簡單（Keeping Neural Networks Simple）
## Hinton & Van Camp (1993) + 現代剪枝技術

### 網路剪枝與壓縮

關鍵洞察：移除不必要的權重以獲得更簡單、更具泛化能力的網路。更小 = 更好！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 用於分類的簡單神經網路

In [ ]:
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

class SimpleNN:
    """簡單的 2 層神經網路"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # 初始化權重
        self.W1 = np.random.randn(input_dim, hidden_dim) * 0.1
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, output_dim) * 0.1
        self.b2 = np.zeros(output_dim)
        
        # 追蹤剪枝的遮罩
        self.mask1 = np.ones_like(self.W1)
        self.mask2 = np.ones_like(self.W2)
    
    def forward(self, X):
        """前向傳遞"""
        # 應用遮罩（針對被剪枝的權重）
        W1_masked = self.W1 * self.mask1
        W2_masked = self.W2 * self.mask2
        
        # 隱藏層
        self.h = relu(np.dot(X, W1_masked) + self.b1)
        
        # 輸出層
        logits = np.dot(self.h, W2_masked) + self.b2
        probs = softmax(logits)
        
        return probs
    
    def predict(self, X):
        """預測類別標籤"""
        probs = self.forward(X)
        return np.argmax(probs, axis=1)
    
    def accuracy(self, X, y):
        """計算準確率"""
        predictions = self.predict(X)
        return np.mean(predictions == y)
    
    def count_parameters(self):
        """計算總參數和活躍（未剪枝）參數數量"""
        total = self.W1.size + self.b1.size + self.W2.size + self.b2.size
        active = int(np.sum(self.mask1) + self.b1.size + np.sum(self.mask2) + self.b2.size)
        return total, active

# 測試網路
nn = SimpleNN(input_dim=10, hidden_dim=20, output_dim=3)
X_test = np.random.randn(5, 10)
y_test = nn.forward(X_test)
print(f"網路輸出形狀：{y_test.shape}")
total, active = nn.count_parameters()
print(f"參數：{total} 總計, {active} 活躍")

## 生成合成資料集

In [ ]:
def generate_classification_data(n_samples=1000, n_features=20, n_classes=3):
    """
    生成合成分類資料集
    每個類別是一個高斯團
    """
    X = []
    y = []
    
    samples_per_class = n_samples // n_classes
    
    for c in range(n_classes):
        # 此類別的隨機中心
        center = np.random.randn(n_features) * 3
        
        # 在中心周圍生成樣本
        X_class = np.random.randn(samples_per_class, n_features) + center
        y_class = np.full(samples_per_class, c)
        
        X.append(X_class)
        y.append(y_class)
    
    X = np.vstack(X)
    y = np.concatenate(y)
    
    # 打亂
    indices = np.random.permutation(len(X))
    X = X[indices]
    y = y[indices]
    
    return X, y

# 生成資料
X_train, y_train = generate_classification_data(n_samples=1000, n_features=20, n_classes=3)
X_test, y_test = generate_classification_data(n_samples=300, n_features=20, n_classes=3)

print(f"訓練集：{X_train.shape}, {y_train.shape}")
print(f"測試集：{X_test.shape}, {y_test.shape}")
print(f"類別分佈：{np.bincount(y_train)}")

## 訓練基準網路

In [ ]:
def train_network(model, X_train, y_train, X_test, y_test, epochs=100, lr=0.01):
    """
    簡單訓練迴圈
    """
    train_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        # 前向傳遞
        probs = model.forward(X_train)
        
        # 交叉熵損失
        y_one_hot = np.zeros((len(y_train), model.output_dim))
        y_one_hot[np.arange(len(y_train)), y_train] = 1
        loss = -np.mean(np.sum(y_one_hot * np.log(probs + 1e-8), axis=1))
        
        # 反向傳遞（簡化版）
        batch_size = len(X_train)
        dL_dlogits = (probs - y_one_hot) / batch_size
        
        # W2, b2 的梯度
        dL_dW2 = np.dot(model.h.T, dL_dlogits)
        dL_db2 = np.sum(dL_dlogits, axis=0)
        
        # W1, b1 的梯度
        dL_dh = np.dot(dL_dlogits, (model.W2 * model.mask2).T)
        dL_dh[model.h <= 0] = 0  # ReLU 導數
        dL_dW1 = np.dot(X_train.T, dL_dh)
        dL_db1 = np.sum(dL_dh, axis=0)
        
        # 更新權重（僅在遮罩活躍的位置）
        model.W1 -= lr * dL_dW1 * model.mask1
        model.b1 -= lr * dL_db1
        model.W2 -= lr * dL_dW2 * model.mask2
        model.b2 -= lr * dL_db2
        
        # 追蹤指標
        train_losses.append(loss)
        test_acc = model.accuracy(X_test, y_test)
        test_accuracies.append(test_acc)
        
        if (epoch + 1) % 20 == 0:
            print(f"輪次 {epoch+1}/{epochs}, 損失：{loss:.4f}, 測試準確率：{test_acc:.2%}")
    
    return train_losses, test_accuracies

# 訓練基準模型
print("訓練基準網路...\n")
baseline_model = SimpleNN(input_dim=20, hidden_dim=50, output_dim=3)
train_losses, test_accs = train_network(baseline_model, X_train, y_train, X_test, y_test, epochs=100)

baseline_acc = baseline_model.accuracy(X_test, y_test)
total_params, active_params = baseline_model.count_parameters()
print(f"\n基準：{baseline_acc:.2%} 準確率, {active_params} 參數")

## 基於幅度的剪枝

移除絕對值最小的權重

In [ ]:
def prune_by_magnitude(model, pruning_rate):
    """
    剪枝幅度最小的權重
    
    pruning_rate: 要移除的權重比例 (0-1)
    """
    # 收集所有權重
    all_weights = np.concatenate([model.W1.flatten(), model.W2.flatten()])
    all_magnitudes = np.abs(all_weights)
    
    # 找到閾值
    threshold = np.percentile(all_magnitudes, pruning_rate * 100)
    
    # 建立新遮罩
    model.mask1 = (np.abs(model.W1) > threshold).astype(float)
    model.mask2 = (np.abs(model.W2) > threshold).astype(float)
    
    print(f"剪枝閾值：{threshold:.6f}")
    print(f"已剪枝 {pruning_rate:.1%} 的權重")
    
    total, active = model.count_parameters()
    print(f"剩餘參數：{active}/{total} ({active/total:.1%})")

# 測試剪枝
import copy
pruned_model = copy.deepcopy(baseline_model)

print("剪枝前：")
acc_before = pruned_model.accuracy(X_test, y_test)
print(f"準確率：{acc_before:.2%}\n")

print("剪枝 50% 的權重...")
prune_by_magnitude(pruned_model, pruning_rate=0.5)

print("\n剪枝後（重新訓練前）：")
acc_after = pruned_model.accuracy(X_test, y_test)
print(f"準確率：{acc_after:.2%}")
print(f"準確率下降：{(acc_before - acc_after):.2%}")

## 剪枝後的微調

重新訓練剩餘權重以恢復準確率

In [ ]:
print("微調剪枝後的網路...\n")
finetune_losses, finetune_accs = train_network(
    pruned_model, X_train, y_train, X_test, y_test, epochs=50, lr=0.005
)

acc_finetuned = pruned_model.accuracy(X_test, y_test)
total, active = pruned_model.count_parameters()

print(f"\n{'='*60}")
print("結果：")
print(f"{'='*60}")
print(f"基準：      {baseline_acc:.2%} 準確率, {total_params} 參數")
print(f"剪枝 50%：  {acc_finetuned:.2%} 準確率, {active} 參數")
print(f"壓縮：      {total_params/active:.1f}x 更小")
print(f"準確率變化：{(acc_finetuned - baseline_acc):+.2%}")
print(f"{'='*60}")

## 漸進式剪枝

逐步增加剪枝率

In [ ]:
def iterative_pruning(model, X_train, y_train, X_test, y_test, 
                     target_sparsity=0.9, num_iterations=5):
    """
    漸進式剪枝和微調
    """
    results = []
    
    # 初始狀態
    total, active = model.count_parameters()
    acc = model.accuracy(X_test, y_test)
    results.append({
        'iteration': 0,
        'sparsity': 0.0,
        'active_params': active,
        'accuracy': acc
    })
    
    # 逐步增加稀疏度
    for i in range(num_iterations):
        # 此輪的稀疏度
        current_sparsity = target_sparsity * (i + 1) / num_iterations
        
        print(f"\n輪次 {i+1}/{num_iterations}：目標稀疏度 {current_sparsity:.1%}")
        
        # 剪枝
        prune_by_magnitude(model, pruning_rate=current_sparsity)
        
        # 微調
        train_network(model, X_train, y_train, X_test, y_test, epochs=30, lr=0.005)
        
        # 記錄結果
        total, active = model.count_parameters()
        acc = model.accuracy(X_test, y_test)
        results.append({
            'iteration': i + 1,
            'sparsity': current_sparsity,
            'active_params': active,
            'accuracy': acc
        })
    
    return results

# 執行漸進式剪枝
iterative_model = copy.deepcopy(baseline_model)
results = iterative_pruning(iterative_model, X_train, y_train, X_test, y_test, 
                           target_sparsity=0.95, num_iterations=5)

## 視覺化剪枝結果

In [ ]:
# 提取資料
sparsities = [r['sparsity'] for r in results]
accuracies = [r['accuracy'] for r in results]
active_params = [r['active_params'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 準確率 vs 稀疏度
ax1.plot(sparsities, accuracies, 'o-', linewidth=2, markersize=10, color='steelblue')
ax1.axhline(y=baseline_acc, color='red', linestyle='--', linewidth=2, label='基準')
ax1.set_xlabel('稀疏度（剪枝比例）', fontsize=12)
ax1.set_ylabel('測試準確率', fontsize=12)
ax1.set_title('準確率 vs 稀疏度', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)
ax1.set_ylim([0, 1])

# 參數 vs 準確率
ax2.plot(active_params, accuracies, 's-', linewidth=2, markersize=10, color='darkgreen')
ax2.axhline(y=baseline_acc, color='red', linestyle='--', linewidth=2, label='基準')
ax2.set_xlabel('活躍參數數', fontsize=12)
ax2.set_ylabel('測試準確率', fontsize=12)
ax2.set_title('準確率 vs 模型大小', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)
ax2.set_ylim([0, 1])
ax2.invert_xaxis()  # 參數較少在右邊

plt.tight_layout()
plt.show()

print("\n關鍵觀察：可以移除 90%+ 的權重，準確率損失極小！")

## 視覺化權重分佈

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 基準權重
axes[0, 0].hist(baseline_model.W1.flatten(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('基準 W1 分佈', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('權重值')
axes[0, 0].set_ylabel('頻率')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(baseline_model.W2.flatten(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('基準 W2 分佈', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('權重值')
axes[0, 1].set_ylabel('頻率')
axes[0, 1].grid(True, alpha=0.3)

# 剪枝後的權重（僅活躍的）
pruned_W1 = iterative_model.W1[iterative_model.mask1 > 0]
pruned_W2 = iterative_model.W2[iterative_model.mask2 > 0]

axes[1, 0].hist(pruned_W1.flatten(), bins=50, color='darkgreen', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('剪枝後 W1 分佈（僅活躍權重）', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('權重值')
axes[1, 0].set_ylabel('頻率')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(pruned_W2.flatten(), bins=50, color='darkgreen', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('剪枝後 W2 分佈（僅活躍權重）', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('權重值')
axes[1, 1].set_ylabel('頻率')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("剪枝後的權重具有較大的幅度（小權重已被移除）")

## 視覺化稀疏模式

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# W1 稀疏模式
im1 = ax1.imshow(iterative_model.mask1.T, cmap='RdYlGn', aspect='auto', interpolation='nearest')
ax1.set_xlabel('輸入維度', fontsize=12)
ax1.set_ylabel('隱藏維度', fontsize=12)
ax1.set_title('W1 稀疏模式（綠色=活躍，紅色=剪枝）', fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=ax1)

# W2 稀疏模式
im2 = ax2.imshow(iterative_model.mask2.T, cmap='RdYlGn', aspect='auto', interpolation='nearest')
ax2.set_xlabel('隱藏維度', fontsize=12)
ax2.set_ylabel('輸出維度', fontsize=12)
ax2.set_title('W2 稀疏模式（綠色=活躍，紅色=剪枝）', fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

total, active = iterative_model.count_parameters()
print(f"\n最終稀疏度：{(total - active) / total:.1%}")
print(f"壓縮比：{total / active:.1f}x")

## MDL 原則

最小描述長度（Minimum Description Length）：更簡單的模型泛化更好

In [ ]:
def compute_mdl(model, X_train, y_train):
    """
    簡化的 MDL 計算
    
    MDL = 模型成本 + 資料成本
    - 模型成本：編碼權重所需的位元
    - 資料成本：編碼誤差所需的位元
    """
    # 模型成本：參數數量（簡化版）
    total, active = model.count_parameters()
    model_cost = active  # 每個參數 = 1 個「位元」（簡化）
    
    # 資料成本：交叉熵損失
    probs = model.forward(X_train)
    y_one_hot = np.zeros((len(y_train), model.output_dim))
    y_one_hot[np.arange(len(y_train)), y_train] = 1
    data_cost = -np.sum(y_one_hot * np.log(probs + 1e-8))
    
    total_cost = model_cost + data_cost
    
    return {
        'model_cost': model_cost,
        'data_cost': data_cost,
        'total_cost': total_cost
    }

# 比較不同模型的 MDL
baseline_mdl = compute_mdl(baseline_model, X_train, y_train)
pruned_mdl = compute_mdl(iterative_model, X_train, y_train)

print("MDL 比較：")
print(f"{'='*60}")
print(f"{'模型':<20} {'模型成本':<15} {'資料成本':<15} {'總計'}")
print(f"{'-'*60}")
print(f"{'基準':<20} {baseline_mdl['model_cost']:<15.0f} {baseline_mdl['data_cost']:<15.2f} {baseline_mdl['total_cost']:.2f}")
print(f"{'剪枝 (95%)':<20} {pruned_mdl['model_cost']:<15.0f} {pruned_mdl['data_cost']:<15.2f} {pruned_mdl['total_cost']:.2f}")
print(f"{'='*60}")
print(f"\n剪枝後的模型總成本更低 → 更好的泛化能力！")

## 關鍵要點

### 神經網路剪枝：

**核心理念**：移除不必要的權重以建立更簡單、更小的網路

### 基於幅度的剪枝：

1. **訓練**網路至正常水準
2. **識別**低幅度權重：$|w| < \text{閾值}$
3. **移除**這些權重（設為 0，遮罩掉）
4. **微調**剩餘權重

### 漸進式剪枝：

比一次性剪枝更好：
```
for 輪次 in 1..N:
    剪枝一小部分（例如 20%）
    微調
```

讓網路逐漸適應。

### 典型結果：

- **50% 稀疏度**：通常沒有準確率損失
- **90% 稀疏度**：輕微準確率損失（<2%）
- **95%+ 稀疏度**：明顯下降

現代網路（ResNets、Transformers）通常可以剪枝到 **90-95% 稀疏度**而影響極小！

### MDL 原則：

$$
\text{MDL} = \underbrace{L(\text{模型})}_{\text{複雜度}} + \underbrace{L(\text{資料 | 模型})}_{\text{誤差}}
$$

**奧卡姆剃刀**：能擬合資料的最簡單解釋（最小網路）是最好的。

### 剪枝的好處：

1. **更小的模型**：更少記憶體，更快推論
2. **更好的泛化**：移除過擬合參數
3. **能源效率**：更少運算
4. **可解釋性**：更簡單的結構

### 剪枝類型：

| 類型 | 移除什麼 | 加速 |
|------|---------|------|
| **非結構化** | 個別權重 | 低（稀疏運算） |
| **結構化** | 整個神經元/濾波器 | 高（密集運算） |
| **通道** | 整個通道 | 高 |
| **層** | 整層 | 非常高 |

### 關鍵洞察：

**神經網路嚴重過度參數化！**

大多數權重對最終效能貢獻很小。剪枝揭示了做實際工作的「核心」網路。

**「最好的模型是能擬合資料的最簡單模型」** - MDL 原則